In [1]:
import os
import sys
from pathlib import Path
import sqlite3 as sql
import pandas as pd

In [2]:
BASE_DIR = Path.cwd().parent
DB_PATH = BASE_DIR / "data" / "portugal_squad.db"
SRC_DIR = BASE_DIR / "src"
sys.path.append(str(BASE_DIR))
# print(f"Project Root: {BASE_DIR}")
# print(f"Database Path: {DB_PATH}")
# print(f"src path: {SRC_DIR}")

In [3]:
# MAKE sql connection
conn = sql.connect(DB_PATH)
query_forward = 'SELECT * FROM forward_base'
query_mid = 'SELECT * FROM midfield_base_new'
query_def = 'SELECT * FROM defense_base'
query_gk  = 'SELECT * FROM goalkeeper_base'

query_all = 'SELECT * FROM players'
query_full_stats = "SELECT * FROM all_player_base"



In [4]:
df_fw = pd.read_sql_query(query_forward,conn)
df_mid = pd.read_sql_query(query_mid,conn)
df_def = pd.read_sql_query(query_def,conn)
df_gk = pd.read_sql_query(query_gk,conn)
df_all = pd.read_sql_query(query_all , conn)

df_full_stats = pd.read_sql_query(query_full_stats,conn)

conn.close()


In [5]:
# create a function to select the euro 2024 as our primary metric target
season_name = 'Euro 2024'

def select_a_season(df:pd.DataFrame,season_name_keywords)-> pd.DataFrame:

    # df['priority'] = df['season_name'].apply(lambda x: 1 if season_name in x else 2)
    df['priority'] = df['season_name'].apply(lambda x: 1 if all(k in str(x) for k in season_name_keywords) else 2)
    # print(df.head())
    # first pivot the table
    df_wide = df.pivot_table(index=['name','season_name','position','priority'],columns='stat_name',values='per_90',aggfunc='first').reset_index()
    df_wide.columns.name = None
    # return df_wide
    df_wide  = df_wide[df_wide['priority']==1]
    df_wide= df_wide.drop(columns=['priority'])

    return df_wide.reset_index(drop=True)



In [6]:
df_all_stats = select_a_season(df_full_stats,['Last','365','UCL'])
# df_all_stats

In [7]:
# df_fw[df_fw['name']=='Cristiano Ronaldo']['season_name'].unique()
df_all['name'].unique()

array(['João Cancelo', 'Renato Veiga', 'Cristiano Ronaldo',
       'Matheus Nunes', 'Vitinha', 'João Palhinha', 'Rúben Neves',
       'Bernardo Silva', 'António Silva', 'Gonçalo Inácio', 'Diogo Dalot',
       'Nélson Semedo', 'José Sá', 'Rui Silva', 'Bruno Fernandes',
       'Rúben Dias', 'Rafael Leão', 'João Neves', 'Carlos Forbs',
       'João Félix', 'Gonçalo Ramos', 'Francisco Conceição'], dtype=object)

In [8]:
df_fw_next = select_a_season(df_fw,['Last','365','Next'])
df_fw_next

,name,season_name,position,Progressive Passes Rec,Shot-Creating Actions,Shots Total,Successful Take-Ons,Touches (Att Pen),npxG: Non-Penalty xG,xAG: Exp. Assisted Goals
0,Carlos Forbs,Last 365 Days Men's Next 14 Competitions,FW-MF,15.72,4.93,1.55,1.41,6.55,0.16,0.21


In [9]:
df_fw_prem = select_a_season(df_fw,['2022','2023','Premier'])
df_fw_prem.head()

,name,season_name,position,Progressive Passes Rec,Shot-Creating Actions,Shots Total,Successful Take-Ons,Touches (Att Pen),npxG: Non-Penalty xG,xAG: Exp. Assisted Goals
0,Cristiano Ronaldo,2022-2023 Premier League,FW-MF (WM),5.66,1.71,4.29,0.69,5.49,0.33,0.07
1,João Félix,2022-2023 Premier League,FW-MF,7.56,2.96,4.40,2.10,4.40,0.47,0.05


In [10]:
df_fw_ucl = select_a_season(df_fw,['Last','365','UCL'])
df_mid_ucl = select_a_season(df_mid,['Last','365','UCL'])
df_def_ucl = select_a_season(df_def,['Last','365','UCL'])
# df_fw_ucl.head()

In [11]:
df_fw_ucl.head()

,name,season_name,position,Progressive Passes Rec,Shot-Creating Actions,Shots Total,Successful Take-Ons,Touches (Att Pen),npxG: Non-Penalty xG,xAG: Exp. Assisted Goals
0,Francisco Conceição,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",FW-MF (AM,8.08,4.60,2.88,2.27,5.89,0.25,0.12
1,Gonçalo Ramos,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",FW,5.02,2.16,4.13,0.52,6.90,0.76,0.10
2,João Félix,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",FW-MF,6.40,4.42,3.57,1.79,5.08,0.29,0.13
3,Rafael Leão,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",FW-MF (AM,8.86,3.13,2.46,1.62,5.92,0.28,0.30


In [12]:
df_mid_ucl.head(6)

,name,season_name,position,Interceptions,Pass Completion %,Progressive Carries,Progressive Passes,Progressive Passes Rec,Shot-Creating Actions,xAG: Exp. Assisted Goals
0,Bernardo Silva,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",FW-MF (AM-CM-WM),0.72,88.4%,2.77,4.43,5.23,2.21,0.11
1,Bruno Fernandes,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",FW-MF (AM-CM-DM),0.63,76.7%,2.12,9.83,3.5,6.35,0.29
2,João Neves,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),1.03,92.5%,1.46,5.69,3.04,2.94,0.14
3,João Palhinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),1.04,83.9%,0.57,3.26,0.8,1.32,0.03
4,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),0.81,93.4%,2.58,9.23,1.46,3.95,0.13


In [13]:
df_def_ucl.head(10)

,name,season_name,position,Aerials Won,Assists,Blocks,Clearances,Interceptions,Progressive Passes,Tackles
0,António Silva,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF (CB,2.51,0.00,1.10,7.42,0.70,2.31,0.90
1,Diogo Dalot,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF-MF (FB),1.72,0.12,0.68,3.15,0.89,3.92,1.60
2,Gonçalo Inácio,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF (CB),2.19,0.00,1.64,3.97,0.68,5.07,0.82
3,Matheus Nunes,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF-MF (CM-DM),0.91,0.17,1.05,3.26,1.13,5.24,2.04
4,Nélson Semedo,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF-MF (DM-FB,0.56,0.05,0.75,2.47,0.93,3.50,1.35
5,Renato Veiga,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF-MF,1.77,0.03,1.41,5.60,1.02,4.06,0.88
6,Rúben Dias,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF-MF (CB),2.01,0.00,1.14,4.08,0.85,4.88,0.62


# optimization for best 11 portugese squad
We start with simple linear programming where we maximize one objective function subject to different constraints, mainly budget and formation

Now for each player we have to create an aggreage score, but before that, we have to normalize the score from the available pool 

In [14]:
def z_scores(df_og):
    df_temp = df_og.copy()
    for col in df_temp.columns:
        if df_temp[col].dtype == 'object': # Only check string-like columns
            # Check if any value contains '%'
            if df_temp[col].astype(str).str.contains('%').any():
                df_temp[col] = df_temp[col].astype(str).str.replace('%', '')
                df_temp[col] = pd.to_numeric(df_temp[col], errors='coerce')            
    mean = df_temp.iloc[:,3:].mean(axis=0)
    std = df_temp.iloc[:,3:].std(axis=0) 
    z_score = (df_temp.iloc[:,3:]-mean)/std 
    z_score['score'] = z_score.mean(axis=1)
    final_df = pd.concat([df_temp[['name','position']],z_score],axis=1)
    return final_df
    


In [15]:
df_fw_zscore = z_scores(df_fw_ucl)
df_mid_zscore = z_scores(df_mid_ucl)
df_def_zscore = z_scores(df_def_ucl)
df_def_zscore

,name,position,Aerials Won,Assists,Blocks,Clearances,Interceptions,Progressive Passes,Tackles,score
0,António Silva,DF (CB,1.209495,-0.785871,-0.029488,1.848353,-1.145267,-1.766955,-0.535780,-0.172216
1,Diogo Dalot,DF-MF (FB),0.075850,0.998269,-1.268001,-0.664029,0.026429,-0.212421,0.838734,-0.029310
2,Gonçalo Inácio,DF (CB),0.750297,-0.785871,1.562885,-0.181557,-1.268603,0.897961,-0.692867,0.040320
3,Matheus Nunes,DF-MF (CM-DM),-1.086495,1.741661,-0.176930,-0.599307,1.506467,1.062104,1.702713,0.592888
4,Nélson Semedo,DF-MF (DM-FB,-1.588743,-0.042480,-1.061582,-1.064127,0.273102,-0.617951,0.347836,-0.536278
5,Renato Veiga,DF-MF,0.147599,-0.339836,0.884652,0.777502,0.828116,-0.077244,-0.575051,0.235105
6,Rúben Dias,DF-MF (CB),0.491998,-0.785871,0.088465,-0.116835,-0.220244,0.714506,-1.085585,-0.130509


create a final table with all players and their position and agg score


In [16]:
cols_to_select =  ['name','position','score']
player_zscore  = pd.concat([df_fw_zscore[cols_to_select],df_mid_zscore[cols_to_select],df_def_zscore[cols_to_select]])
                        
player_zscore

,name,position,score
0,Francisco Conceição,FW-MF (AM,0.113271
1,Gonçalo Ramos,FW,-0.078792
2,João Félix,FW-MF,-0.124418
3,Rafael Leão,FW-MF (AM,0.089938
0,Bernardo Silva,FW-MF (AM-CM-WM),0.037018
1,Bruno Fernandes,FW-MF (AM-CM-DM),0.322491
2,João Neves,MF (CM-DM),0.137122
3,João Palhinha,MF (CM-DM),-0.763652
4,Vitinha,MF (CM-DM),0.267021
0,António Silva,DF (CB,-0.172216


In [17]:
def extract_roles(position):
    roles = []
    if 'FW' in position:
        roles.append('FW')
    if 'MF' in position:
        roles.append('MF')
    if 'DF' in position:
        roles.append('DF')
    if 'GK' in position:
        roles.append('GK')
    return roles

def extract_global_role(position):
    roles = []
    if 'FW' in position and not (('CM') or ('DM') or ('WM')) in position :
        roles.append('FW')
    if 'MF' in position and not ('DF') in position:
        roles.append('MF')
    if 'DF' in position:
        roles.append('DF')
    if 'GK' in position:
        roles.append('GK')
    
    return roles[0]

In [18]:
# player_zscore['roles'] = player_zscore['position'].apply(extract_one_role)
# players_zscore = player_zscore.to_dict(orient='records')
# players_zscore

In [19]:
# from src.milp_solver import SquadMILPSolver

# formation = (4,3,3)


# sq_milp_zscore = SquadMILPSolver(player_info=players_zscore,formation=formation,total_players=10,role_aware=False)

# results = sq_milp_zscore.solve()

# df_results_zscore = pd.DataFrame(results['selected_players'])

# print("Players selected with Z-score")
# df_results_zscore


# PCA for feature extraction

In [20]:
df_players = df_all_stats.copy()

In [21]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np 

In [22]:
for col in df_players.columns:
        if df_players[col].dtype == 'object': # Only check string-like columns
            # Check if any value contains '%'
            if df_players[col].astype(str).str.contains('%').any():
                df_players[col] = df_players[col].astype(str).str.replace('%', '')
                df_players[col] = pd.to_numeric(df_players[col], errors='coerce')

In [23]:
df_players.columns

Index(['name', 'season_name', 'position', 'Aerials Won', 'Assists', 'Blocks',
       'Clearances', 'Interceptions', 'Non-Penalty Goals', 'Pass Completion %',
       'Passes Attempted', 'Progressive Carries', 'Progressive Passes',
       'Progressive Passes Rec', 'Shot-Creating Actions', 'Shots Total',
       'Successful Take-Ons', 'Tackles', 'Touches (Att Pen)', 'npxG + xAG',
       'npxG: Non-Penalty xG', 'xAG: Exp. Assisted Goals'],
      dtype='object')

In [24]:
x = df_players.iloc[:,3:]
# standardize 
x_scaled = StandardScaler().fit_transform(x)
#  pca 
pca = PCA(n_components=3)
pc = pca.fit_transform(x_scaled)
df_pca = pd.DataFrame(data = pc, columns = ['PC_1', 'PC_2', 'PC_3'])
df_pca.insert(0,'name',df_players['name'].values)
df_pca.insert(1,'position',df_players['position'].values)

features =  df_players.columns[3:]
loadings = pd.DataFrame(pca.components_.T, columns=['PC_1', 'PC_2', 'PC_3'], index=features)
loadings
# df_pca

,PC_1,PC_2,PC_3
Aerials Won,-0.157694,-0.341537,-0.200742
Assists,0.083938,0.369620,-0.369104
Blocks,-0.243087,0.038127,-0.165772
Clearances,-0.262069,-0.186057,0.058280
Interceptions,-0.260493,0.187251,-0.032407
Non-Penalty Goals,0.260130,-0.092155,-0.329446
Pass Completion %,-0.225785,0.148730,0.009253
Passes Attempted,-0.174011,0.365549,-0.030557
Progressive Carries,0.199930,0.162511,0.458708
Progressive Passes,-0.018940,0.463321,-0.127562


In [25]:
role_scores = {'CM':'PC_2','AM':'PC_2','DM':'PC_1'}

def get_role_scores(df):
    # Define role score formulas
    df_tmp = df.copy()
    role_formulas = {
        'CF': lambda row: row['PC_1'],
        'AM': lambda row: 0.5 * row['PC_1'] + row['PC_2'],
        'CM': lambda row: row['PC_2'],
        'DM': lambda row: -row['PC_1'] - row['PC_2'],
        'CB': lambda row: -row['PC_1'],
        'WM': lambda row: row['PC_3'],
        'FB': lambda row: -0.5 * row['PC_1'] + row['PC_3'],
    }
    
    # Calculate scores for each player's sub roles
    role_scores = []
    for _, row in df.iterrows():
        player_scores = {role: role_formulas[role](row) 
                        for role in row['roles'] 
                        if role in role_formulas}
        role_scores.append(player_scores)
    
    df_tmp['score'] = role_scores
    df_tmp = df_tmp.drop(columns=['PC_1','PC_2','PC_3'])
    return df_tmp



In [26]:
df_pca['global role'] = df_pca['position'].apply(extract_global_role)
df_pca

,name,position,PC_1,PC_2,PC_3,global role
0,António Silva,DF (CB,-3.639031,-2.968208,0.118884,DF
1,Bernardo Silva,FW-MF (AM-CM-WM),-0.490335,0.894827,1.054112,MF
2,Bruno Fernandes,FW-MF (AM-CM-DM),2.411448,3.374569,-2.288230,MF
3,Diogo Dalot,DF-MF (FB),0.129990,-0.210388,0.555370,DF
4,Francisco Conceição,FW-MF (AM,4.092953,0.090246,2.700020,FW
5,Gonçalo Inácio,DF (CB),-3.324164,-0.637184,0.276186,DF
6,Gonçalo Ramos,FW,4.913235,-3.716815,-2.852499,FW
7,João Félix,FW-MF,3.805468,-0.743134,1.066216,FW
8,João Neves,MF (CM-DM),-0.604337,1.461641,-1.325403,MF
9,João Palhinha,MF (CM-DM),-2.286751,-0.131475,-2.050572,MF


In [27]:
import re 
def extract_positions(pos_string):

    # Look for content after '(' regardless of closing bracket
    bracket_match = re.search(r'\(([^)]+)', pos_string)
    
    if bracket_match:
        # Extract and clean positions inside/after bracket
        positions = bracket_match.group(1).strip().split('-')
        # Remove any remaining whitespace
        positions = [p.strip() for p in positions]
    else:
        # No brackets - use fallback mapping
        clean_pos = pos_string.strip()
        
        position_mapping = {
            'FW': ['CF'],
            'MF': ['CM'],  # or ['CM', 'AM'] if you want both
            'DF': ['CB'],
            'DF-MF': ['DM','CB'],
            'FW-MF': ['AM', 'CF'],
        }
        
        positions = position_mapping.get(clean_pos, clean_pos.split('-'))
    
    return positions

In [28]:
df_pca['roles'] = df_pca['position'].apply(extract_positions)
df_pca

,name,position,PC_1,PC_2,PC_3,global role,roles
0,António Silva,DF (CB,-3.639031,-2.968208,0.118884,DF,[CB]
1,Bernardo Silva,FW-MF (AM-CM-WM),-0.490335,0.894827,1.054112,MF,"[AM, CM, WM]"
2,Bruno Fernandes,FW-MF (AM-CM-DM),2.411448,3.374569,-2.288230,MF,"[AM, CM, DM]"
3,Diogo Dalot,DF-MF (FB),0.129990,-0.210388,0.555370,DF,[FB]
4,Francisco Conceição,FW-MF (AM,4.092953,0.090246,2.700020,FW,[AM]
5,Gonçalo Inácio,DF (CB),-3.324164,-0.637184,0.276186,DF,[CB]
6,Gonçalo Ramos,FW,4.913235,-3.716815,-2.852499,FW,[CF]
7,João Félix,FW-MF,3.805468,-0.743134,1.066216,FW,"[AM, CF]"
8,João Neves,MF (CM-DM),-0.604337,1.461641,-1.325403,MF,"[CM, DM]"
9,João Palhinha,MF (CM-DM),-2.286751,-0.131475,-2.050572,MF,"[CM, DM]"


In [29]:
# 
df_pca_role_scores = get_role_scores(df_pca)
df_pca_role_scores
# df_pca_role_scores.to_csv(f"{BASE_DIR}/data/squad_roles_scores.csv")

,name,position,global role,roles,score
0,António Silva,DF (CB,DF,[CB],{'CB': 3.6390314776543855}
1,Bernardo Silva,FW-MF (AM-CM-WM),MF,"[AM, CM, WM]","{'AM': 0.6496601563736771, 'CM': 0.89482747938..."
2,Bruno Fernandes,FW-MF (AM-CM-DM),MF,"[AM, CM, DM]","{'AM': 4.580292639355582, 'CM': 3.374568886519..."
3,Diogo Dalot,DF-MF (FB),DF,[FB],{'FB': 0.49037489681575985}
4,Francisco Conceição,FW-MF (AM,FW,[AM],{'AM': 2.1367223064611816}
5,Gonçalo Inácio,DF (CB),DF,[CB],{'CB': 3.324163831503056}
6,Gonçalo Ramos,FW,FW,[CF],{'CF': 4.913235015168584}
7,João Félix,FW-MF,FW,"[AM, CF]","{'AM': 1.1595996391651293, 'CF': 3.80546750245..."
8,João Neves,MF (CM-DM),MF,"[CM, DM]","{'CM': 1.4616408126862284, 'DM': -0.8573036739..."
9,João Palhinha,MF (CM-DM),MF,"[CM, DM]","{'CM': -0.1314751815522355, 'DM': 2.4182259177..."


In [30]:
# make a final table with player info 
# test case 1 : pc1 as the agg score
player_info = df_pca.copy().drop(columns=['PC_1','PC_3']).rename(columns={'PC_2':'score'})
player_info  = player_info.to_dict(orient='records')
# player_info

# Feature extraction with PC axes:
looking at PC loadings we can safely infer the following:
1. PC1 = FW score (seperates attackers/finishers vs defenders)
2. PC2 = MF score (seperates strong midfields (CM) vs defenders)
3. PC3 =  wingers/ (WM/LW/RW)
4. -PC1 = defenders (FB,CB)
5. -PC1-PC2  = DM (defensive midfielders)

we can make any combinations for the respective roles, for example, pure ST or poacher could be PC1 - PC2, AM = 0.5.PC1+PC2, CM= PC2, WM = 0.5.PC1+ PC3,  but for simplicity we only take the given 5 format and this could be easily extended once we have numerous players and more advanced features for the optimization model.

**now these roles are soft-constraints as we know these positions are more fluid during the game. So the hard constraints still be on 3 main categories, FW(CF,AM,ST) , MF (CM,DM),DF(FB,CB) ...**


In [31]:
df_pca_role_scores

,name,position,global role,roles,score
0,António Silva,DF (CB,DF,[CB],{'CB': 3.6390314776543855}
1,Bernardo Silva,FW-MF (AM-CM-WM),MF,"[AM, CM, WM]","{'AM': 0.6496601563736771, 'CM': 0.89482747938..."
2,Bruno Fernandes,FW-MF (AM-CM-DM),MF,"[AM, CM, DM]","{'AM': 4.580292639355582, 'CM': 3.374568886519..."
3,Diogo Dalot,DF-MF (FB),DF,[FB],{'FB': 0.49037489681575985}
4,Francisco Conceição,FW-MF (AM,FW,[AM],{'AM': 2.1367223064611816}
5,Gonçalo Inácio,DF (CB),DF,[CB],{'CB': 3.324163831503056}
6,Gonçalo Ramos,FW,FW,[CF],{'CF': 4.913235015168584}
7,João Félix,FW-MF,FW,"[AM, CF]","{'AM': 1.1595996391651293, 'CF': 3.80546750245..."
8,João Neves,MF (CM-DM),MF,"[CM, DM]","{'CM': 1.4616408126862284, 'DM': -0.8573036739..."
9,João Palhinha,MF (CM-DM),MF,"[CM, DM]","{'CM': -0.1314751815522355, 'DM': 2.4182259177..."


In [32]:
player_stats_roles = df_pca_role_scores.to_dict(orient='records')
form = (4,3,3)

In [33]:
from src.milp_solver import SquadMILPSolver

sq_roles = SquadMILPSolver(player_stats_roles,formation=form,total_players=10,role_aware=True)
results_role = sq_roles.solve()

TypeError: SquadMILPSolver.__init__() missing 4 required positional arguments: 'age', 'total_budget', 'playing_style', and 'locked_players'

In [ ]:
df_= pd.DataFrame(results_role['selected_players'])

df_

,name,role
0,António Silva,CB
1,Bernardo Silva,WM
2,Bruno Fernandes,AM
3,Gonçalo Inácio,CB
4,Gonçalo Ramos,CF
5,João Félix,CF
6,Nélson Semedo,FB
7,Renato Veiga,DM
8,Rúben Dias,CB
9,Vitinha,CM
